In [ ]:
import os
import pathlib
%load_ext autoreload
%autoreload 2

In [ ]:
def is_integer(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

In [ ]:
root = pathlib.Path("/storage/vbutoi/datasets/WMH")
splits = ["training", "test", "additional_annotations"]

all_dirs = []

for split in splits:
    split_path = root / split
    for subdir in split_path.iterdir():
        print(subdir)
        all_dirs.append(subdir)
        for l3_dir in subdir.iterdir():
            if not is_integer(str(l3_dir.name)):
                print(l3_dir)
                all_dirs.append(l3_dir)
                for l4_dir in l3_dir.iterdir():
                    if not is_integer(str(l4_dir.name)):
                        print(l4_dir)
                        all_dirs.append(l4_dir)
                        for l5_dir in l4_dir.iterdir():
                            if not is_integer(str(l5_dir.name)):
                                print(l5_dir)
                                all_dirs.append(l5_dir)

In [ ]:
unique_dirs_with_additional = []
for path in all_dirs:
    all_other_dirs = [p for p in all_dirs if p != path]
    is_subdir = False
    for other_path in all_other_dirs:
        if path in other_path.parents:
            is_subdir = True
            break
    if not is_subdir:
        unique_dirs_with_additional.append(path)

In [ ]:
unique_dirs_with_additional

In [ ]:
unique_dirs = [ud for ud in unique_dirs_with_additional if "additional_annotations" not in str(ud)]

In [ ]:
unique_dirs

In [ ]:
import nibabel as nib
import nibabel.processing as nip
import numpy as np
from PIL import Image

def resample_nib(img, voxel_spacing=(1, 1, 1), order=3):
    """Resamples the nifti from its original spacing to another specified spacing
    
    Parameters:
    ----------
    img: nibabel image
    voxel_spacing: a tuple of 3 integers specifying the desired new spacing
    order: the order of interpolation
    
    Returns:
    ----------
    new_img: The resampled nibabel image 
    
    """
    # resample to new voxel spacing based on the current x-y-z-orientation
    aff = img.affine
    shp = img.shape
    zms = img.header.get_zooms()
    # Calculate new shape
    new_shp = tuple(np.rint([
        shp[0] * zms[0] / voxel_spacing[0],
        shp[1] * zms[1] / voxel_spacing[1],
        shp[2] * zms[2] / voxel_spacing[2]
        ]).astype(int))
    new_aff = nib.affines.rescale_affine(aff, shp, voxel_spacing, new_shp)
    new_img = nip.resample_from_to(img, (new_shp, new_aff), order=order, cval=-1024)
    return new_img


def resample_mask_to(msk, to_img):
    """Resamples the nifti mask from its original spacing to a new spacing specified by its corresponding image
    
    Parameters:
    ----------
    msk: The nibabel nifti mask to be resampled
    to_img: The nibabel image that acts as a template for resampling
    
    Returns:
    ----------
    new_msk: The resampled nibabel mask 
    
    """
    to_img.header['bitpix'] = 8
    to_img.header['datatype'] = 2  # uint8
    new_msk = nib.processing.resample_from_to(msk, to_img, order=0)
    return new_msk


def pad_image_numpy(image_array, target_width, target_height):
    # Convert the NumPy array to a Pillow image
    original_image = Image.fromarray(image_array)

    # Get the original dimensions
    original_width, original_height = original_image.size

    # Calculate the amount of padding needed
    width_padding = target_width - original_width
    height_padding = target_height - original_height

    # Calculate the padding borders
    left_padding = width_padding // 2
    right_padding = width_padding - left_padding
    top_padding = height_padding // 2
    bottom_padding = height_padding - top_padding

    # Get the border pixel value
    border_pixel = original_image.getpixel((10, 10))

    # Create a new image with the desired dimensions and fill with border pixel
    padded_image = Image.new(original_image.mode, (target_width, target_height), border_pixel)

    # Paste the original image onto the padded image with padding borders
    padded_image.paste(original_image, (left_padding, top_padding))

    # Convert the padded image back to a NumPy array
    padded_image_array = np.array(padded_image)

    return padded_image_array

def normalize_image(image):
    # Convert the image to floating point format
    image = image.astype(np.float32)

    # Normalize the image between 0 and 1
    normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))
    
    return normalized_image

In [9]:
import sys
sys.path.append("/storage/vbutoi/projects/ESE")
sys.path.append("/storage/vbutoi/projects")
from ese.experiment.datasets.utils.build_wmh import proc_WMH

proc_WMH(
    unique_dirs, 
    modalities=["FLAIR"],
    show=True,
    save=False
    )

KeyboardInterrupt: 

In [ ]:
from thunderpack import ThunderDB

def thunderify_WMH(datacenter, modality, version=0.1):

    source_dir = pathlib.Path("/storage/vbutoi/datasets/WMH/thunder")
    with ThunderDB.open(source_dir + 'data', 'c') as db:
        db['metadata'] = {
            'version': version, 
            'n_samples': None
        }
        
        keys = []
        for i in range(100):
            key = f'sample{i:02d}'
            x = np.random.normal(size=(128,128))
            y = np.random.normal(size=(128,128))
            # Thunderpack will serialize the tuple and numpy arrays automatically
            db[key] = (x, y) 
            keys.append(key)

        db['samples'] = keys